<a href="https://colab.research.google.com/github/leoninekev/ml-engine-custom-prediction-routine/blob/master/custom_prediciton_model_deployment_mlengine_Walkthrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## Custom prediction routines 

**Model deployment task in google Ai-latform necessitates compliance, completion of following three sub-rotines:**
* Upload of model artifacts to cloud storage bucket (Artifacts include - saved model/keras Model file, custom prediction, pre/post processing scripts, confi files)
* Creation of ai-platform **Model Resource**.
* Creation of ai-platform **Version Resource** (Specifying path to cloud storage holding Model & artifact package)

###  Custom prediction routines determines, what code runs when an online prediction request to ai-platform is made.

Deploying a custom prediction routine as **Version Resource** serves many utilities as follows:
* It enables AI Platform to run a custom python code in response to each incoming request received for prediction/inferencing.
* It allows preprocessing of input data before it is forwarded to a trained model for prediction.
* It also allows postprocessing model's output prior to actual posting of prediction result, thus modifying output to suit any application's endpoints requirements.

In [0]:
PROJECT_ID = "nifty-episode-231612" #@param {type:"string"}
! gcloud config set project $PROJECT_ID

In [0]:
import sys

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS ''


In [2]:
!git clone https://github.com/leoninekev/ml-engine-custom-prediction-routine.git

Cloning into 'ml-engine-custom-prediction-routine'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 28 (delta 9), reused 24 (delta 9), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [4]:
%cd ml-engine-custom-prediction-routine/
! ls -pR

/content/ml-engine-custom-prediction-routine
.:
config.pickle		    predictor.py  roi_helpers.py
FixedBatchNormalization.py  README.md	  RoiPoolingConv.py
LICENSE			    resnet.py	  setup.py


In [5]:
!ls

config.pickle		    predictor.py  roi_helpers.py
FixedBatchNormalization.py  README.md	  RoiPoolingConv.py
LICENSE			    resnet.py	  setup.py


* View the setup file and edit the package name and version if needed.

In [0]:
%pycat setup.py

* Packaging the Predictor module and other files

In [6]:
!python setup.py sdist --formats=gztar

running sdist
running egg_info
creating test_code.egg-info
writing test_code.egg-info/PKG-INFO
writing dependency_links to test_code.egg-info/dependency_links.txt
writing requirements to test_code.egg-info/requires.txt
writing top-level names to test_code.egg-info/top_level.txt
writing manifest file 'test_code.egg-info/SOURCES.txt'
writing manifest file 'test_code.egg-info/SOURCES.txt'
running check


creating test_code-0.1
creating test_code-0.1/test_code.egg-info
copying files to test_code-0.1...
copying FixedBatchNormalization.py -> test_code-0.1
copying LICENSE -> test_code-0.1
copying README.md -> test_code-0.1
copying RoiPoolingConv.py -> test_code-0.1
copying config.pickle -> test_code-0.1
copying predictor.py -> test_code-0.1
copying resnet.py -> test_code-0.1
copying roi_helpers.py -> test_code-0.1
copying setup.py -> test_code-0.1
copying test_code.egg-info/PKG-INFO -> test_code-0.1/test_code.egg-info
copying test_code.egg-info/SOURCES.txt -> test_code-0.1/test_code.egg-info


* Now the copy the packaged python modules or artifacts to the cloud storage bucket path where model weights files are stored.

In [8]:
!ls

config.pickle		    LICENSE	  resnet.py	     setup.py
dist			    predictor.py  roi_helpers.py     test_code.egg-info
FixedBatchNormalization.py  README.md	  RoiPoolingConv.py
